# Eat Safe, Love

## Notebook Set Up

In [12]:
# Import dependencies
import pymongo
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [13]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [14]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [15]:
# review the collections in our database
collections = db.list_collection_names()
print("Collections List:")
for collection in collections:
    print(collection)
print()

Collections List:
establishments



In [16]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [17]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene":20}

# Use count_documents to display the number of documents in the result
doc_count = db.establishments.count_documents(query)
print("Number of documents with hygiene score of 20: ",doc_count)
print(" ")

# Display the first document in the results using pprint
query = establishments.find_one({"scores.Hygiene": 20})

# Display the first document using pprint
pprint(query)

Number of documents with hygiene score of 20:  41
 
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('645d70fb30b57b694e17917b'),
 'geocode': {'latitude': 50.769705, 'longitude': 0.27694},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],


In [18]:
# Convert the result to a Pandas DataFrame
# Perform the query and retrieve the results
query = establishments.find({"scores.Hygiene": 20})
results = list(query)

# Extract the desired fields from the results
hygiene_data = []
for result in results:
    hygiene_data.append({
        "BusinessName": result["BusinessName"],
        "BusinessType": result["BusinessType"],
        "AddressLine1": result["AddressLine1"],
        "LocalAuthorityName": result["LocalAuthorityName"],
        "Hygiene": result["scores"]["Hygiene"]
    })

# Create a DataFrame 
hygiene_df = pd.DataFrame(hygiene_data)

# Display the first 10 rows
display(hygiene_df.head(10))
print(" ")
print("Number of rows in DataFrame: ", hygiene_df.shape[0])

,BusinessName,BusinessType,AddressLine1,LocalAuthorityName,Hygiene
0,The Chase Rest Home,Caring Premises,5-6 Southfields Road,Eastbourne,20
1,Brenalwood,Caring Premises,Hall Lane,Tendring,20
2,Melrose Hotel,Hotel/bed & breakfast/guest house,53 Marine Parade East,Tendring,20
3,Seaford Pizza,Takeaway/sandwich shop,4 High Street,Lewes,20
4,Golden Palace,Restaurant/Cafe/Canteen,5 South Street,Lewes,20
5,Ashby's Butchers,Retailers - other,777 Southchurch Road,Southend-On-Sea,20
6,South Sea Express Cuisine,Restaurant/Cafe/Canteen,33 Alexandra Street,Southend-On-Sea,20
7,Golden Palace,Takeaway/sandwich shop,7 London Road,Rochford,20
8,The Tulip Tree,Restaurant/Cafe/Canteen,3 The Village,Sevenoaks,20
9,F & S,Retailers - other,,Basildon,20


 
Number of rows in DataFrame:  41


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [19]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {
    'AddressLine4': {'$regex': 'London', '$options': 'i'},
    'RatingValue': {'$gte': 4}
}

# Count the number of documents in the result
doc_count = db.establishments.count_documents(query)
print("Number of documents for establishments in London with a rating greater than or equal to 4:", doc_count)

# Find and display the first document in the results using find_one
query = db.establishments.find_one(query)
pprint(query)

Number of documents for establishments in London with a rating greater than or equal to 4: 390
{'AddressLine1': 'Arch D',
 'AddressLine2': 'The Link Thamesmead',
 'AddressLine3': 'Bazalgette Way',
 'AddressLine4': 'London',
 'BusinessName': 'The Mezzanine Cafe',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4646.357884390823,
 'FHRSID': 1049722,
 'LocalAuthorityBusinessID': '18/00096/CP',
 'LocalAuthorityCode': '503',
 'LocalAuthorityEmailAddress': 'food.safety@bexley.gov.uk',
 'LocalAuthorityName': 'Bexley',
 'LocalAuthorityWebSite': 'http://www.bexley.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE2 9BS',
 'RatingDate': '2022-05-24T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': 4,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('645d70fd30b57b694e17e62e'),
 'geocode': {'latitude': 51.5002632141113, 'longitude': 0.121751002967358},
 'links': [{'href': 'http://api.ratings.food.gov.uk/es

In [20]:
# Execute the query and store the results in the query variable
query = db.establishments.find({
    'AddressLine4': {'$regex': 'London', '$options': 'i'},
    'RatingValue': {'$gte': 4}
})

# Convert the result to a Pandas DataFrame
results = list(query)

# Extract the desired fields from the results
London_rating_data = []
for result in results:
    London_rating_data.append({
        "BusinessName": result["BusinessName"],
        "BusinessType": result["BusinessType"],
        "AddressLine1": result["AddressLine1"],
        "AddressLine4": result["AddressLine4"],
        "RatingValue": result["RatingValue"]
    })

# Create a DataFrame 
London_rating_df = pd.DataFrame(London_rating_data)

# Display the first 10 rows
London_rating_df.head(10)

,BusinessName,BusinessType,AddressLine1,AddressLine4,RatingValue
0,The Mezzanine Cafe,Restaurant/Cafe/Canteen,Arch D,London,4
1,"St Edward's Church, Mottingham - StEP Lunch Cl...",Other catering premises,St Edwards Church Hall,London,4
2,Maryfield Pre-School 2,Caring Premises,Mottingham Community Centre,London,5
3,Co-op,Retailers - supermarkets/hypermarkets,The Co-operative Food,London,5
4,North Hub @ Castlecombe Youth Centre,Caring Premises,Castlecombe Youth Centre,London,4
5,Spencers,Retailers - other,11 Cranley Parade,London,4
6,Domino's Pizza,Takeaway/sandwich shop,3 Cranley Parade,London,5
7,Papa John's Pizza,Takeaway/sandwich shop,4 Cranley Parade,London,5
8,Perfect Cafe,Restaurant/Cafe/Canteen,11 Beaconsfield Parade,London,5
9,Superfry Fish Bar,Takeaway/sandwich shop,12 Beaconsfield Parade,London,4


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [21]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

# query for penang flavours document
penang_flavours_query = {
    'BusinessName': 'Penang Flavours'
}
penang_flavours_result = db.establishments.find_one(penang_flavours_query)

# set degree search to 0.01; query to find lon and lat.
degree_search = 0.01
latitude = penang_flavours_result['geocode']['latitude']
longitude = penang_flavours_result['geocode']['longitude']

#Write query to find restaurants within 0.1 degrees of Penang Flavours with a rating value of 5
query = {
    'geocode.latitude': {'$gte': latitude - degree_search, '$lte': latitude + degree_search},
    'geocode.longitude': {'$gte': longitude - degree_search, '$lte': longitude + degree_search},
    'RatingValue': 5
}
#sort according to hygiene scores
sort = [('scores.Hygiene', pymongo.ASCENDING)]

ranked_results = db.establishments.find(query).sort(sort).limit(5)

# Retrieve the results from the cursor
data = list(ranked_results)

# Print the results
ranked_results_list =[]
for result in data:
    ranked_results_list.append(result)
    pprint(result)

{'AddressLine1': 'Restaurant And Premises 107A Plumstead High Street',
 'AddressLine2': '',
 'AddressLine3': 'Plumstead',
 'AddressLine4': 'Greenwich',
 'BusinessName': 'Howe and Co Fish and Chips - Van 17',
 'BusinessType': 'Mobile caterer',
 'BusinessTypeID': 7846,
 'ChangesByServerID': 0,
 'Distance': 4646.95593107927,
 'FHRSID': 1380578,
 'LocalAuthorityBusinessID': '14425',
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'SE18 1SE',
 'RatingDate': '2021-11-11T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('645d70fd30b57b694e17e9ba'),
 'geocode': {'latitude': 51.4875335693359, 'longitude': 0.0925370007753372},
 'links': [{'href': 'http://api.ratings.food.gov.uk/establishments/1380578',
            'rel': 'self'}],


In [22]:
# #create data frame from query results

# Set desired columns
columns = ['BusinessName', 'BusinessType', 'Hygiene', 'RatingValue', 'longitude', 'latitude']

# Create an empty DataFrame
ranked_results_df = pd.DataFrame(columns=columns)

# Iterate over the list and append the desired values to the DataFrame
for result in ranked_results_list:
    row = {
        'BusinessName': result['BusinessName'],
        'BusinessType': result['BusinessType'],
        'Hygiene': result['scores']['Hygiene'],
        'RatingValue': result['RatingValue'],
        'longitude': result['geocode']['longitude'],
        'latitude': result['geocode']['latitude']
    }
    ranked_results_df = ranked_results_df.append(row, ignore_index=True)

#Display DataFrame
ranked_results_df

/var/folders/5d/5wkwj7dn70dc_4kvq7_g5vnm0000gn/T/ipykernel_32419/2230978756.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_results_df = ranked_results_df.append(row, ignore_index=True)
/var/folders/5d/5wkwj7dn70dc_4kvq7_g5vnm0000gn/T/ipykernel_32419/2230978756.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_results_df = ranked_results_df.append(row, ignore_index=True)
/var/folders/5d/5wkwj7dn70dc_4kvq7_g5vnm0000gn/T/ipykernel_32419/2230978756.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_results_df = ranked_results_df.append(row, ignore_index=True)
/var/folders/5d/5wkwj7dn70dc_4kvq7_g5vnm0000gn/T/ipykernel_32419/2230978756.py:19: FutureWarning: The frame.append method is deprecated and

,BusinessName,BusinessType,Hygiene,RatingValue,longitude,latitude
0,Howe and Co Fish and Chips - Van 17,Mobile caterer,0,5,0.092537,51.487534
1,Atlantic Fish Bar,Takeaway/sandwich shop,0,5,0.091216,51.486730
2,Plumstead Manor Nursery,Caring Premises,0,5,0.085994,51.481518
3,Iceland,Retailers - supermarkets/hypermarkets,0,5,0.092420,51.487148
4,Volunteer,Pub/bar/nightclub,0,5,0.092080,51.487344


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [23]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results

# Create the pipeline
pipeline = [
    {'$match': {'scores.Hygiene': 0}},
    {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
]

# Execute the pipeline
result = list(db.establishments.aggregate(pipeline))

# Print the number of documents in the result
print("Number of documents in result: ", len(result))

# Print the first 10 results
for doc in result[:10]:
    pprint(doc)


Number of documents in result:  55
{'_id': 'Thanet', 'count': 1130}
{'_id': 'Greenwich', 'count': 882}
{'_id': 'Maidstone', 'count': 713}
{'_id': 'Newham', 'count': 711}
{'_id': 'Swale', 'count': 686}
{'_id': 'Chelmsford', 'count': 680}
{'_id': 'Medway', 'count': 672}
{'_id': 'Bexley', 'count': 607}
{'_id': 'Southend-On-Sea', 'count': 586}
{'_id': 'Tendring', 'count': 542}


In [24]:
# Convert the result to a DataFrame
zero_hygiene_df = pd.DataFrame(result)
zero_hygiene_df = zero_hygiene_df.rename(columns={'_id': 'LocalAuthorityName'})

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(zero_hygiene_df))

# Display the first 10 rows of the DataFrame
print("Number of restaurants with a hygiene of zero by local authority:")
zero_hygiene_df.head(10)

Number of rows in the DataFrame: 55
Number of restaurants with a hygiene of zero by local authority:


,LocalAuthorityName,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
